# How to use *iMAT* with *Troppo*

A typical workflow follows two main steps. The first is to attribute a score to each reaction of the model, in accordance with the omics data imputed. The second is to use the scores and apply an integration method to select a subset of reactions to build the final model.

Integration scoring methods implemented in *Troppo* are:
- continuous: `ContinuousScoreIntegrationStrategy`
- threshold: `ThresholdSelectionIntegrationStrategy`
- default_core: `DefaultCoreIntegrationStrategy`
- adjusted_score: `AdjustedScoreIntegrationStrategy`
- custom: `CustomSelectionIntegrationStrategy`

Omics integration methods implemented in *Troppo* are:
- gimme: `GIMME`
- tinit: `tINIT`
- fastcore: `fastCORE`
- imat: `IMAT`
- swiftcore: `SWIFTCORE`
- corda: `CORDA`

This example can be applied for all the Omics integration methods implemented in this package. Note that the appropriate integration scoring method can differ between integration algorithms. For instance, for *GIMME* a continuous scoring method can be used, while for `fastcore` a threshold scoring method is more adequate.

### Imports and Setup

In [1]:
import pandas as pd
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.reconstruction.imat import IMAT, IMATProperties

C:\Users\biosy\anaconda3\envs\troppo\lib\site-packages\cobamp-0.2.1-py3.8.egg\cobamp\wrappers\external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(


Define the parsing rules for the GPRs that will be used later on.

In [2]:
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

### Read model and omics data

In [3]:
model = cobra.io.read_sbml_model('data/HumanGEM_Consistent_COVID19_HAM.xml')
model

Name,HumanGEM
Memory address,1e5bb993280
Number of metabolites,6149
Number of reactions,10347
Number of genes,2976
Number of groups,142
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Lysosome, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [4]:
omics_data = pd.read_csv(filepath_or_buffer='data/Desai-GTEx_ensembl.csv', index_col=0)

### Create a container for the omics data.

The `TabularReader` class is used to read and store the omics data in a container that can then be used by *Troppo*. 

Relevant arguments from the `TabularReader` class:
- `path_or_df`: the omics data can be either a pandas dataframe or a path to a dataset file. The file can be in any format supported by pandas.
- `index_col`: the name of the column that contains the identifiers of the genes.
- `sample_in_rows`: a boolean indicating whether the samples are in rows or columns.
- `header_offset`: the number of rows to skip before reading the header.
- `omics_type`: a string containing the type of omics data. This is used to select the appropriate integration method.
- `nomenclature`: a string containing the nomenclature of the identifiers in the omics data. This is used to map the identifiers to the identifiers in the model.

The `to_containers()` method returns a list of containers, one for each sample of the dataset. In this example, we will be using only one sample, however, the process can be iterated for all the samples in the dataset.
The `get_integrated_data_map()` method is used to map the identifiers in the omics data to the identifiers in the model. This is done by using the `gpr_gene_parse_function` argument from the `ModelBasedWrapper` class.

In [5]:
omics_container = TabularReader(path_or_df=omics_data, nomenclature='ensemble_gene_id', omics_type='transcriptomics').to_containers()[0]
omics_container

### Create a model wrapper.

The `ModelBasedWrapper` class is used to wrap the model so that it can be used by *Troppo*.

Relevant arguments from this class include:
- `model`: the model to be wrapped.
- `ttg_ratio`: the ratio between the number of reactions to be selected and the total number of reactions in the model.
- `gpr_gene_parse_function`: a function that parses the GPRs of the model. This is used to map the identifiers in the omics data to the identifiers in the model.

Important attributes from this class include:
- `model_reader`: a COBRAModelObjectReader instance containing all the information of the model, such as, reaction_ids, metabolite_ids, GPRs, bounds, etc.
- `S`: the stoichiometric matrix of the model.
- `lb`: the lower bounds of the reactions in the model.
- `ub`: the upper bounds of the reactions in the model.

In [6]:
model_wrapper = ModelBasedWrapper(model=model, ttg_ratio=9999, gpr_gene_parse_function=replace_alt_transcripts)
model_wrapper

C:\Users\biosy\anaconda3\envs\troppo\lib\site-packages\cobamp-0.2.1-py3.8.egg\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000131069
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\troppo\lib\site-packages\cobamp-0.2.1-py3.8.egg\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111058 or _ENSG00000154930
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\troppo\lib\site-packages\cobamp-0.2.1-py3.8.egg\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000091140 and _ENSG00000110435 and _ENSG00000131828 and _ENSG00000150768 and _ENSG00000163114 and _ENSG00000168291
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\troppo\lib\site-packages\cobamp-0.2.1-py3.8.egg\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111716 or _ENSG00000134333 or _ENSG00000151116 or _ENSG00000

### Map the identifiers in the omics data to the identifiers in the model

For this we can use the `get_integrated_data_map()` method from the `TabularReader` class. This maps the gene ids in the omics dataset reaction ids in the model through their GPRs, and attributes a score to each reaction in accordance with the expression values of the associated genes. This method returns a dictionary with the reaction ids as keys and the scores as values.

Important arguments from this method include:
- `model_reader`: a COBRAModelObjectReader instance containing all the information of the model. It can be accessed through the `model_wrapper.model_reader`.
- `and_func`: a function that is used to combine the scores of the genes associated with a reaction for AND rules in the GPR. In this example, we will be using the minimum function, which means that the score of a reaction with AND in their GPRs will be the minimum score of the genes associated with it.
- `or_func`: a function that is used to combine the scores of the genes associated with a reaction for OR rules in the GPR. In this example, we will be using the sum function, which means that the score of a reaction with OR in their GPRs will be the sum of the scores of the genes associated with it.

In [7]:
data_map = omics_container.get_integrated_data_map(model_reader=model_wrapper.model_reader, and_func=min, or_func=sum)

### Integrate scores

The `integrate()` method from the `ContinuousScoreIntegrationStrategy` class is used to integrate the scores of the reactions in the model. This method returns a dictionary with the reaction ids as keys and the integrated scores as values. In the case of this continuous scoring method, the resulting scores are the same as the scores in the data map. However, for other scoring methods, such as threshold scoring methods, the result will be a list of reactions with a score above the selected threshold. 

Moreover, this method allows us to apply an additional function to the method, which can be useful if you have any protected reactions that need to be in the final model or to remove nan values from the result. This can be done by passing the function as the `score_apply` argument of the `ContinuousScoreIntegrationStrategy` class. 

In this example, we will be using a function that replaces the nan values with 0 and returns a list with all the scores. This is the required format for the *iMAT* method.

In [8]:
def score_apply(reaction_map_scores):
    return [0 if v is None else v for k, v in reaction_map_scores.items()]

continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply)
scores = continuous_integration.integrate(data_map=data_map)

print(scores)

[3.336283388, 7.301587646, 3.336283388, 3.336283388, 0.0, 16.317904841999997, 16.317904841999997, 10.289723824, 24.161542437999998, 14.972071356, 14.972071356, 12.94899207, 9.398209262, 0, 17.701664925, 12.370940183, 11.741142932999999, 8.185172776, 12.370940183, 3.765534746, 10.17729495, 12.94899207, 7.230933093, 6.704595348, 0, 20.330185487, 12.078519108000002, 5.649615459, 5.169925001, 3.201633861, 3.5360529, 16.472204492, 3.857980995, 6.704595348, 6.704595348, 0, 6.012121673, 10.412659603, 11.44493205, 4.336283388, 12.084110784, 1.201633861, 1.201633861, 12.084110784, 1.584962501, 0, 4.350497247, 5.070389328, 3.070389328, 3.070389328, 6.606442228000001, 0.0, 14.972071356, 14.972071356, 0, 0.0, 10.334977139, 0.0, 6.541483864, 3.070389328, 0, 12.850353762, 0, 6.541483864, 15.986777665000002, 4.078951341, 13.568817585999998, 14.238585064999999, 10.322108421, 3.485426827, 0, 2.925999419, 0, 15.986777665000002, 13.615888508, 0, 6.17990909, 5.626439136, 6.541483864, 0, 0, 0, 6.490249211,

### Run the iMAT algorithm

The `IMATProperties` class is used to create the properties for the iMAT algorithm. This class contains the following arguments:
- `exp_vector`: a list of scores for each reaction in the model. This can be obtained from the `integrate()` method of the `ContinuousScoreIntegrationStrategy` class.
- `exp_thresholds`: The thresholds for the expression values.
- `core`: The core reactions, by default None.
- `tolerance`: The tolerance, by default 1e-8.
- `epsilon`: The epsilon, by default 1.

The `IMAT` class is used to run the iMAT algorithm. This class contains the following arguments:
- `S`: the stoichiometric matrix of the model. It can be accessed through the `model_wrapper.S`.
- `lb`: the lower bounds of the reactions in the model. It can be accessed through the `model_wrapper.lb`.
- `ub`: the upper bounds of the reactions in the model. It can be accessed through the `model_wrapper.ub`.
- `properties`: a `IMATProperties` instance containing the properties for the iMAT algorithm.

In the end, the `run()` method of the `IMAT` class will return a list of zeros, ones, and twos indicating the reactions that should be removed or kept to be in the final model. This list can be used to build the final model.

Moreover, to access the flux distribution determined by the algorithm, you can use the `sol` attribute of the `IMAT` class.

In [ ]:
# Create the properties for the IMAT algorithm.
properties = IMATProperties(exp_vector=scores, exp_thresholds=(0.2,0.5))

# Run the iMAT algorithm.
imat = IMAT(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

model_imat = imat.run()

print(model_imat)

Could not set parameters with this solver
